In [14]:
import sys
import os
import pickle
import pandas as pd
import json
import datetime
import time
import copy

py_file_location = '../PrivatePackages'
sys.path.append(os.path.abspath(py_file_location))
from sklearn.model_selection import train_test_split

import JiaoCheng
import NingXiang

In [15]:
LABEL = 'trend'
MODEL = 'lgbc'

In [16]:
curated_data = pd.read_csv('../data/curated_data.csv')
curated_data = curated_data.drop(columns=['closePrice', 'closePriceNorm', 'ret', 'log_ret', 'stock'])
curated_data['date'] = pd.to_datetime(curated_data['date'])

In [17]:
curated_data

,date,trend,ret_lag_1,log_ret_lag_1,closePrice_lag_1,closePriceNorm_lag_1,ret_lag_2,log_ret_lag_2,closePrice_lag_2,closePriceNorm_lag_2,...,closePrice_lag_8,closePriceNorm_lag_8,ret_lag_9,log_ret_lag_9,closePrice_lag_9,closePriceNorm_lag_9,ret_lag_10,log_ret_lag_10,closePrice_lag_10,closePriceNorm_lag_10
0,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-02,-1.0,NaN,NaN,60.17,-0.982328,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-03,-1.0,0.997008,-0.002996,59.99,-1.094387,NaN,NaN,60.17,-0.982328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-04,-1.0,0.994499,-0.005516,59.66,-1.299827,0.997008,-0.002996,59.99,-1.094387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-05,-1.0,0.995307,-0.004704,59.38,-1.474140,0.994499,-0.005516,59.66,-1.299827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,2023-09-03,-1.0,0.991745,-0.008289,43.25,0.244890,0.998855,-0.001146,43.61,0.508653,...,42.40,-0.377885,0.987128,-0.012955,42.18,-0.539073,0.987748,-0.012327,42.73,-0.136102
12496,2023-09-04,1.0,0.994682,-0.005332,43.02,0.076374,0.991745,-0.008289,43.25,0.244890,...,42.42,-0.363231,1.005216,0.005202,42.40,-0.377885,0.987128,-0.012955,42.18,-0.539073
12497,2023-09-05,1.0,1.000232,0.000232,43.03,0.083701,0.994682,-0.005332,43.02,0.076374,...,42.47,-0.326598,1.000472,0.000472,42.42,-0.363231,1.005216,0.005202,42.40,-0.377885
12498,2023-09-06,-1.0,1.012549,0.012471,43.57,0.479346,1.000232,0.000232,43.03,0.083701,...,42.52,-0.289964,1.001179,0.001178,42.47,-0.326598,1.000472,0.000472,42.42,-0.363231


In [18]:
date = [datetime.datetime(year=2023, month=1, day = 1) + datetime.timedelta(days=i) for i in range(250)]
val_date_start = date[175]
test_date_start = date[175+38]

In [19]:
for i in range(1, 11):
    print('Max Lag:', i)

    tmp_curated_data = copy.deepcopy(curated_data)

    for j in range(i+1, 11):
        tmp_curated_data = tmp_curated_data.drop(columns=[f'log_ret_lag_{j}', f'ret_lag_{j}'], axis = 1)
    
    tmp_curated_data = tmp_curated_data.dropna()

    train = tmp_curated_data[(tmp_curated_data['date'] < val_date_start)]
    val = tmp_curated_data[(tmp_curated_data['date'] < test_date_start) & (tmp_curated_data['date'] >= val_date_start)]

    ningxiang = NingXiang.NingXiang()

    train_x = train.drop(columns=['date', 'trend'], axis = 1)
    train_y = train['trend']

    val_x = val.drop(columns=['date', 'trend'], axis = 1)
    val_y = val['trend']

    ningxiang.read_in_train_data(train_x, train_y, val_x, val_y)

    ningxiang.set_model_type('Classification')

    ningxiang_output = ningxiang.get_rf_based_feature_combinations(1, 1, 4)

    ningxiang.export_ningxiang_output(f'../models/NingXiang_output_{LABEL}_lag_{i}')

    ningxiang.show_rf_stats()

    print('\n\n')

Max Lag: 1
NingXiang Initialised
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Successfully recorded model type: Classification
Begin fitting Random Forest
Finished fitting Random Forest
Successfully set object output address
{'ret_lag_1': 0.049854433952988136, 'log_ret_lag_1': 0.050189197263175236, 'closePrice_lag_1': 0.02618072848039725, 'closePriceNorm_lag_1': 0.07160290678152581, 'closePrice_lag_2': 0.02334080339092695, 'closePriceNorm_lag_2': 0.06244865180258823, 'closePrice_lag_3': 0.024065294758891843, 'closePriceNorm_lag_3': 0.0685071394052089, 'closePrice_lag_4': 0.024838343045528666, 'closePriceNorm_lag_4': 0.06105171730586971, 'closePrice_lag_5': 0.024198613863823274, 'closePriceNorm_lag_5': 0.05958615578375246, 'closePrice_lag_6': 0.0253497983053434, 'closePriceNorm_lag_6': 0.06796091895584169, 'closePrice_lag_7': 0.024267959835602787, 'closePriceNorm_lag_7': 0.06454108175424522, 'closePrice_lag_8': 0.025593430635991024, 'closePriceNorm_lag

In [24]:
for i in range(1, 11):
    print('Max Lag:', i)

    tmp_curated_data = copy.deepcopy(curated_data)

    for j in range(i+1, 11):
        tmp_curated_data = tmp_curated_data.drop(columns=[f'log_ret_lag_{j}', f'ret_lag_{j}'], axis = 1)
    
    tmp_curated_data = tmp_curated_data.dropna()

    train = tmp_curated_data[(tmp_curated_data['date'] < val_date_start)]
    val = tmp_curated_data[(tmp_curated_data['date'] < test_date_start) & (tmp_curated_data['date'] >= val_date_start)]
    test = tmp_curated_data[(tmp_curated_data['date'] >= test_date_start)]

    jiaocheng = JiaoCheng.JiaoCheng()

    train_x = train.drop(columns=['date', 'trend'], axis = 1)
    train_y = train['trend']

    val_x = val.drop(columns=['date', 'trend'], axis = 1)
    val_y = val['trend']
    
    test_x = test.drop(columns=['date', 'trend'], axis = 1)
    test_y = test['trend']

    with open(f'../models/NingXiang_output_{LABEL}_lag_{i}.pickle', 'rb') as f:
        ningxiang_output = pickle.load(f)

    ningxiang_output = list(ningxiang_output.items())
    ningxiang_output = dict(ningxiang_output)

    jiaocheng = JiaoCheng.JiaoCheng()

    from lightgbm import LGBMClassifier as LGBC

    jiaocheng.read_in_model(LGBC, 'Classification')

    jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

    del train_x
    del train_y
    del val_x
    del val_y

    parameter_choices = {
        'n_estimators': (25, 50, 100, 200, 400),
        'max_depth': (3, 6, 12, 24, 48, 96, 192),
        'subsample': (0.4, 0.55, 0.7, 0.85),
        'colsample_bytree': (0.4, 0.55, 0.7, 0.85),
        'learning_rate': (0.00000001, 0.000001, 0.0001, 0.01, 0.1),
        'reg_alpha': (0.0001, 0.001, 0.01, 0.1, 1, 10)

    }

    jiaocheng.set_hyperparameters(parameter_choices)

    jiaocheng.set_non_tuneable_hyperparameters({'random_state': 19260817, 'n_jobs': -1})

    jiaocheng.set_features(ningxiang_output)

    jiaocheng.set_tuning_order(['features', 'max_depth', 'colsample_bytree', 'subsample', 'n_estimators', 'reg_alpha', 'learning_rate'])

    jiaocheng.set_hyperparameter_default_values({
        'features': 0,
        'n_estimators': 50,
        'max_depth': 12,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'learning_rate': 0.1,
        'reg_alpha': 0.01}
    )

    try:
        jiaocheng.read_in_tuning_result_df(f'../models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    except:
        pass

    jiaocheng.set_tuning_result_saving_address(f'../models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    jiaocheng.set_best_model_saving_address(f'../models/tmp_models/jiaocheng_{MODEL}_{LABEL}_lag_{i}.pickle')

    jiaocheng.tune()

    jiaocheng.view_best_combo_and_score()

    print('\n\n')

Max Lag: 1
JiaoCheng Initialised
JiaoCheng Initialised
Successfully read in model <class 'lightgbm.sklearn.LGBMClassifier'>, which is a Classification model
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
Successfully recorded hyperparameter choices
Successfully recorded non_tuneable_hyperparameter choices
Successfully recorded tuneable feature combination choices and updated relevant internal structures
Successfully read in tuning result of 80 rows
Successfully set tuning output address
Successfully set best model output address

Default combo: [1, 2, 2, 2, 4, 2, 0] 


ROUND 1

Round 1 
Hyperparameter: features (index: 6) 

As new Best Combo (1, 2, 2, 2, 4, 2, 0) was read in, best_clf is set to None
Already Trained and Tested combination (1, 2, 2, 2, 4, 2, 0), which had val score of 0.5253
        Current best combo: (1, 2, 2, 2, 4, 2, 0) with val score 0.5253. 
        Has trained 80 of 369600 combinations so far